In [17]:
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table

In [18]:
decals_bricks = Table.read('survey-bricks-dr10-south.fits')
galaxy_groups = Table.read('asu.fit')

decals_bricks

brickname,ra,dec,nexp_g,nexp_r,nexp_i,nexp_z,nexphist_g,nexphist_r,nexphist_i,nexphist_z,nobjs,npsf,nsimp,nrex,nexp,ndev,ncomp,nser,ndup,psfsize_g,psfsize_r,psfsize_i,psfsize_z,psfdepth_g,psfdepth_r,psfdepth_i,psfdepth_z,galdepth_g,galdepth_r,galdepth_i,galdepth_z,ebv,trans_g,trans_r,trans_i,trans_z,cosky_g,cosky_r,cosky_i,cosky_z,ext_g,ext_r,ext_i,ext_z,wise_nobs,trans_wise,ext_w1,ext_w2,ext_w3,ext_w4,brickid,ra1,ra2,dec1,dec2,area,survey_primary,in_desi
bytes8,float64,float64,int16,int16,int16,int16,int32[11],int32[11],int32[11],int32[11],int32,int32,int32,int32,int32,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16[4],float32[4],float32,float32,float32,float32,int32,float64,float64,float64,float64,float64,bool,bool
0001m002,0.125,-0.25,10,10,7,8,13542 .. 7514096,14702 .. 8386255,25938 .. 0,20589 .. 855903,8737,3442,0,4085,827,220,0,163,0,1.5703166,1.2452925,1.231581,1.1920182,25.22465,25.0845,24.319462,23.908138,24.97652,24.75831,24.007229,23.556026,0.03402216,0.9041929,0.93440855,0.95133764,0.9627636,-7.889145e-05,-5.5482946e-05,-4.7043235e-05,2.3911733e-05,0.10934724,0.073658,0.054163303,0.041200835,210 .. 12,0.99425083 .. 0.9997149,0.00626009,0.0038445229,0.00081992405,0.00030957558,328928,0.0,0.25,-0.375,-0.125,0.062499405045045905,True,True
0001m005,0.125,-0.5,10,10,7,8,933 .. 8548737,2810 .. 8727647,4615 .. 0,2346 .. 1804628,9398,3480,0,4419,1003,299,0,197,0,1.5510331,1.2331864,1.2204007,1.2144874,25.223454,25.088036,24.371918,23.900475,24.976059,24.769518,24.038757,23.562841,0.041127287,0.88537395,0.92126334,0.9414779,0.95516396,-7.551222e-05,-6.0565366e-05,-3.5909343e-05,6.3544736e-05,0.13218315,0.08904053,0.06547468,0.049805187,207 .. 13,0.9930544 .. 0.99965537,0.007567411,0.0046474035,0.000991173,0.00037424616,327488,0.0,0.25,-0.625,-0.375,0.06249762019151071,True,True
0001m007,0.125,-0.75,12,10,7,8,417 .. 10493328,872 .. 6679819,3965 .. 0,1755 .. 556469,9220,3630,0,4273,850,272,0,193,2,1.6269429,1.2314527,1.2185445,1.2110996,25.267267,25.08984,24.330029,23.87891,25.036816,24.759014,24.001347,23.534582,0.04344342,0.87932444,0.9170183,0.93828595,0.95269966,-7.840524e-05,-5.7306017e-05,-2.5664453e-05,1.4516765e-05,0.13962714,0.094055004,0.06916197,0.052609973,208 .. 14,0.9926647 .. 0.99963593,0.00799356,0.004909095,0.0010470096,0.00039535074,326048,0.0,0.25,-0.875,-0.625,0.06249464547337544,True,True
0001m010,0.125,-1.0,12,8,6,7,1327 .. 10747994,3208 .. 1949928,5744 .. 0,3265 .. 323647,9071,3379,0,4334,889,281,0,188,0,1.6708362,1.2558109,1.2354081,1.2016487,25.221338,24.999865,24.184875,23.839066,25.001518,24.680262,23.859194,23.485859,0.038021848,0.8935505,0.9269858,0.9457747,0.95847815,-8.257703e-05,-5.800799e-05,-2.5445392e-05,2.0877764e-05,0.122202225,0.08231729,0.060530797,0.046044454,205 .. 12,0.9935771 .. 0.99968135,0.006996042,0.0042965007,0.00091629685,0.00034602112,324608,0.0,0.25,-1.125,-0.875,0.062490480947274454,True,True
0001m012,0.125,-1.25,13,9,6,9,517 .. 10570804,1323 .. 5576703,3120 .. 0,1429 .. 1635280,8890,3446,0,4155,843,253,0,192,1,1.658045,1.2659082,1.2486696,1.1876932,25.297417,25.067265,24.234285,23.930859,25.07275,24.751226,23.902834,23.58452,0.04097276,0.885779,0.9215472,0.9416913,0.95532864,-8.6032655e-05,-5.430796e-05,-1.748512e-05,1.728364e-05,0.13168652,0.088706076,0.065228626,0.049618002,203 .. 13,0.9930804 .. 0.9996567,0.007538998,0.0046299854,0.0009874809,0.00037282196,323168,0.0,0.25,-1.375,-1.125,0.06248512669249432,True,True
0001m015,0.125,-1.5,11,8,5,8,547 .. 9424192,1993 .. 457946,4333 .. 0,1680 .. 213767,9291,3541,0,4443,865,281,0,161,0,1.6144478,1.2491776,1.2271997,1.1906184,25.246355,25.00759,24.194231,23.882137,25.01422,24.68276,23.861238,23.5358,0.03663752,0.8972197,0.9295482,0.9476964,0.9599592,-8.320551e-05,-5.7625635e-05,-2.2080774e-05,2.346852e-06,0.117752984,0.079320215,0.058326937,0.044368

In [19]:
brick_coords = SkyCoord(ra=decals_bricks['ra'], dec=decals_bricks['dec'], unit=(u.deg, u.deg))

galaxy_coords = SkyCoord(ra=galaxy_groups['RAJ2000'], dec=galaxy_groups['DEJ2000'], unit = (u.hourangle, u.deg))

galaxy_groups['RAJ2000'] = galaxy_coords.icrs.ra.degree
galaxy_groups['DEJ2000'] = galaxy_coords.icrs.dec.degree
galaxy_groups.write('groups.fits', overwrite=True)

In [20]:
galaxy_groups

Group,RAJ2000,DEJ2000,zUV,nUV,ngal,nz,mz,r_theta_,Rij,sigmalos,Simbad
,,,,,,,,arcmin,kpc,km / s,
int16,float64,float64,float64,int16,int16,int16,int16,float64,float64,float64,bytes7
1,0.09554166666666665,-22.58711111111111,0.0,4,4,0,0,0.913,0.0,0.0,Simbad
2,0.34475,13.108916666666666,0.017899,4,4,3,3,1.611,32.94,193.85,Simbad
3,2.8417916666666665,-53.955444444444446,0.041719,4,3,1,1,1.249,0.0,0.0,Simbad
4,3.3276666666666666,-41.579638888888894,0.0,4,3,0,0,1.027,0.0,0.0,Simbad
5,3.537583333333333,-22.56413888888889,0.0,4,4,0,0,1.149,0.0,0.0,Simbad
6,4.443124999999999,-69.14861111111112,0.0,4,3,0,0,1.336,0.0,0.0,Simbad
7,4.619958333333332,-42.12755555555555,0.093258,4,3,1,1,0.891,0.0,0.0,Simbad
8,4.6624583333333325,-35.95266666666667,0.012953,4,4,2,2,0.888,0.0,0.0,Simbad


In [17]:
def check_footprint(decals_bricks, galaxy_groups):
    # Convertir las coordenadas de los bricks en un objeto SkyCoord
    brick_coords = SkyCoord(ra=decals_bricks['ra'], dec=decals_bricks['dec'], unit=(u.deg, u.deg))

    # Convertir las coordenadas de los grupos de galaxias en un objeto SkyCoord
    galaxy_coords = SkyCoord(ra=galaxy_groups['RAJ2000'], dec=galaxy_groups['DEJ2000'], unit = (u.hourangle, u.deg))

    # Verificar si cada grupo de galaxias está dentro de algún brick
    within_footprint = np.zeros(len(galaxy_coords), dtype=bool)

    for i, galaxy_coord in enumerate(galaxy_coords):
        distances = galaxy_coord.separation(brick_coords).to('arcmin')
        if np.any(distances < (15*np.sqrt(2)/2)*u.arcmin):
            within_footprint[i] = True

    return within_footprint

# Ejemplo de uso
#decals_bricks = np.array([
#    [120.0, 30.0],
#    [130.0, 40.0],
    # ... coordenadas RA, DEC de otros bricks ...
#])

#galaxy_groups = np.array([
#    [121.0, 31.0],
#    [135.0, 45.0],
#    # ... coordenadas RA, DEC de otros grupos de galaxias ...
#])



In [18]:
within_footprint = check_footprint(decals_bricks, galaxy_groups)
print(within_footprint)

[ True  True  True  True  True  True  True  True False  True False False
 False  True False False False  True  True  True  True  True  True False
  True False False  True  True  True  True  True  True  True  True  True
  True False  True False False  True  True False  True False  True  True
  True  True  True  True  True  True  True  True  True  True False False
 False False False  True False  True  True  True  True False  True  True
 False  True  True False  True False  True False  True False False False
  True False False  True  True  True  True False False False  True  True
  True  True False  True  True False False  True False False False False
 False False False  True  True False False False  True False False False
 False  True False False  True  True False False False False False False
 False False False  True False  True  True  True  True False  True  True
  True  True  True  True False False False False False  True False  True
  True  True False  True False False False  True  T

In [24]:
#galaxy_groups.add_column(within_footprint, name = 'DECaLS') 

In [25]:
galaxy_groups['RAJ2000'] = galaxy_coords.icrs.ra.degree
galaxy_groups['DEJ2000'] = galaxy_coords.icrs.dec.degree
galaxy_groups

Group,RAJ2000,DEJ2000,zUV,nUV,ngal,nz,mz,r_theta_,Rij,sigmalos,Simbad,DECaLS
,,,,,,,,arcmin,kpc,km / s,,
int16,float64,float64,float64,int16,int16,int16,int16,float64,float64,float64,bytes7,bool
1,0.09554166666666665,-22.58711111111111,0.0,4,4,0,0,0.913,0.0,0.0,Simbad,True
2,0.34475,13.108916666666666,0.017899,4,4,3,3,1.611,32.94,193.85,Simbad,True
3,2.8417916666666665,-53.955444444444446,0.041719,4,3,1,1,1.249,0.0,0.0,Simbad,True
4,3.3276666666666666,-41.579638888888894,0.0,4,3,0,0,1.027,0.0,0.0,Simbad,True
5,3.537583333333333,-22.56413888888889,0.0,4,4,0,0,1.149,0.0,0.0,Simbad,True
6,4.443124999999999,-69.14861111111112,0.0,4,3,0,0,1.336,0.0,0.0,Simbad,True
7,4.619958333333332,-42.12755555555555,0.093258,4,3,1,1,0.891,0.0,0.0,Simbad,True
8,4.6624583333333325,-35.95266666666667,0.012953,4,4,2,2,0.888,0.0,0.0,Simbad,True
